In [2]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
# from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [3]:
# data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, 1818, 1228, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [4]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(1228, 1818, 3))
for layer in vgg.layers:
    layer.trainable = False

In [5]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1228, 1818, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1228, 1818, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1228, 1818, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 614, 909, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 614, 909, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 614, 909, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 307, 454, 128)     0         
__________

In [134]:
p1 = Conv2D(filters=75, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = Conv2D(filters=75, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=17).output)
p3 = Conv2D(filters=75, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p2))
p4 = Conv2D(filters=75, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p3))

p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p12 = Activation('relu')(p12)
p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p22 = Activation('relu')(p22)
p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p3)
p32 = Activation('relu')(p32)
p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p4)
p42 = Activation('relu')(p42)

p12 = K.concatenate(tensors=[p11, p12], axis = -1)
p22 = K.concatenate(tensors=[p21, p22], axis = -1)
p32 = K.concatenate(tensors=[p31, p32], axis = -1)
p42 = K.concatenate(tensors=[p41, p42], axis = -1)

In [135]:
pred = [K.reshape(p12, (-1, 5)), K.reshape(p22, (-1, 5)), K.reshape(p32, (-1, 5)), K.reshape(p42, (-1, 5))]

In [136]:
pred = K.concatenate(pred, axis = 0)

In [146]:
def NMS(boxes = pred):
    
    Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[:, 1:], (-1, 4)), scores=K.reshape(boxes[:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
#     A = K.reshape(K.tf.gather_nd(boxes, Z0), (-1, 5))
    A = K.reshape(K.gather(boxes, Z0), (-1, 5))
    return A

In [138]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y, a=0.8):
    if K.tf.size(Y) != K.tf.size(y):
        n, _ = Y.get_shape().as_list()
        y_ = y[:n]
        
    softMax = K.mean(-(Y[:, 0] * K.log(y_[:, 0]) + (1 - Y[:, 0]) * K.log(1 - y_[:, 0])))
    mse = K.mean(K.square(Y[:, 1:] - y_[:, 1:]))
    return 1.15 * softMax + 0.8 * mse

In [147]:
sess.run(K.tf.global_variables_initializer())

In [152]:
# b = sess.run([NMS()], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis = 0)})[0]
bbox = NMS()

In [153]:
b

array([[0.86912763, 0.9870889 , 3.8230536 , 0.        , 0.03183609],
       [0.8680369 , 0.8033845 , 2.2322636 , 0.        , 0.77447045],
       [0.85129035, 1.6109104 , 3.4727721 , 0.        , 0.41822252],
       [0.8441588 , 0.3285332 , 0.47960415, 0.        , 0.        ],
       [0.84229434, 0.9918303 , 1.3631871 , 0.        , 0.9969671 ],
       [0.83733076, 0.6286531 , 0.26513872, 0.        , 0.        ],
       [0.83260393, 0.        , 2.068152  , 0.        , 0.18978946],
       [0.8294149 , 0.        , 1.10045   , 0.        , 0.00562084],
       [0.8277325 , 0.0758786 , 0.        , 0.        , 0.        ],
       [0.8276129 , 0.05095381, 0.52046585, 0.        , 0.        ],
       [0.8275071 , 0.        , 0.04786368, 0.        , 0.30582076],
       [0.82734483, 0.3486284 , 0.59671146, 0.        , 0.8295694 ],
       [0.8259494 , 1.0271451 , 2.5398088 , 0.        , 0.4660762 ],
       [0.8243294 , 1.44274   , 0.7100208 , 0.        , 0.69356984],
       [0.82265514, 0.        , 0.

In [154]:
Loss = loss(Y, bbox)

In [155]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [156]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
#         for i, d in enumerate(data):
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0), Y: np.ones((2, 5)).reshape((-1, 5))})
        cost += L
        if e % 100 == 0:
            print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [157]:
train(epochs = 12)

ResourceExhaustedError: OOM when allocating tensor with shape[1,512,227,153]
	 [[Node: conv4_3/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv4_2/Relu, conv4_3/kernel/read)]]
	 [[Node: add_9/_465 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1629_add_9", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv4_3/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chawat/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chawa